# ACS/Census Variable web scraping 

## Finding the variable names for Census and ACS data is hard. It can either be incredibly montonous having to go one by one and changing the names via Excel or even using some random R package to convert these things! 

### This script serves to do a couple things: 

#### 1. Reads in the 'Variables' page from Census and ACS tables 
#### 2. Scrapes the table contents and converts them into a Data Frame 
#### 3. The output provides a CSV that can be referenced into a list or a dictionary so that the column names for whatever tables you pull can be easily renamed 

In [1]:
import pandas as pd
import requests 
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re

### Create dataframe out of the 'variables' page offered on ACS/Census website based on the table.

In [2]:
emp_matrix = "https://data.bls.gov/projections/nationalMatrix?queryParams=621000&ioType=i"
varnames_r = requests.get(emp_matrix)
text = varnames_r.text
text_soup = BeautifulSoup(text, 'lxml')

#### Now that we have the web page in a readable format, let's find all of the table cells.  The html tag " td" is how table cells are formatted in html language. 

#### Go the the section starting with 'Table' here to see a complete list of html tags: https://www.w3schools.com/TAGs/

In [3]:
rows = text_soup.find_all("tr")
print(rows[:10])

[<tr>
<th class="oep cellbordered" id="col1" style="text-align: center; vertical-align: bottom"><strong>Occupation Title</strong></th>
<th class="oep cellbordered" id="col2" style="text-align: center; vertical-align: bottom"><strong>Occupation Code</strong></th>
<th class="oep cellbordered" id="col3" style="text-align: center; vertical-align: bottom"><strong>Occupation Type</strong></th>
<th class="oep cellbordered" id="col4" style="text-align: center; vertical-align: bottom"><strong>2018 Employment</strong></th>
<th class="oep cellbordered" id="col5" style="text-align: center; vertical-align: bottom"><strong>2018 Percent of Industry</strong></th>
<th class="oep cellbordered" id="col6" style="text-align: center; vertical-align: bottom"><strong>2018 Percent of Occupation</strong></th>
<th class="oep cellbordered" id="col7" style="text-align: center; vertical-align: bottom"><strong>Projected 2028 Employment</strong></th>
<th class="oep cellbordered" id="col8" style="text-align: center; v

#### So a table exists! We can now run a for loop to gather all of those td cells into a table format.

#### Here is some documentation on re: https://docs.python.org/2/library/re.html

In [4]:
list_rows = []
for row in rows:
    # gets all of the cells with the html tag <td>
    cells = row.find_all('td')
    # converts all of the cells into a string
    str_cells = str(cells)
    # creates a regular expression object so that the string can be looped 
    clean = re.compile('<.*?>')
    # doing a second round of cleaning by substituting the clean cells with the string cells  and then appending
    # into the list `list_rows`
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[Packers and packagers, hand, 53-7064, Line Item, 0.1, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 31.5, 1070, 4]


str

#### Creating a dataframe out of the list of cells that we just created

In [5]:
bls_621_empmat = pd.DataFrame(list_rows)
bls_621_empmat.head(10)

,0
0,[]
1,"[\nFilter by Title:\n\n, \nFilter by Code:\n\n..."
2,"[Total, all occupations, 00-0000, Summary, 7,4..."
3,"[Management occupations, 11-0000, Summary, 223..."
4,"[Top executives, 11-1000, Summary, 49.4, 0.7, ..."
5,"[Chief executives, 11-1011, Line Item, 5.8, 0...."
6,"[General and operations managers, 11-1021, Lin..."
7,"[Advertising, marketing, promotions, public re..."
8,"[Advertising and promotions managers, 11-2011,..."
9,"[Marketing and sales managers, 11-2020, Summar..."


#### Our dataframe is a bit clunky with the way that it's structured because it maintained the form of a list while still being committed to a df. 

#### Use str.split to treat it as a comma delimiter. Documentation: https://python-reference.readthedocs.io/en/latest/docs/str/split.html

In [7]:
df1 = bls_621_empmat[0].str.split(',', expand = True)
df1.head(10)

# Let's also remove the leading bracket from the dataframe to clean things up a bit more.

df1[0] = df1[0].str.strip('[')
df1.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,\nFilter by Title:\n\n,\nFilter by Code:\n\n,,,,,,,,,,,],None,None,None,None
2,Total,all occupations,00-0000,Summary,7,498.9,100.0,4.7,9,383.1,100.0,5.5,1,884.2,25.1,,0]
3,Management occupations,11-0000,Summary,223.7,3.0,2.2,300.6,3.2,2.8,76.8,34.3,2,1],None,None,None,None
4,Top executives,11-1000,Summary,49.4,0.7,1.8,61.8,0.7,2.2,12.4,25.2,3,2],None,None,None,None
5,Chief executives,11-1011,Line Item,5.8,0.1,2.2,6.3,0.1,2.5,0.5,8.7,4,3],None,None,None,None
6,General and operations managers,11-1021,Line Item,43.5,0.6,1.8,55.5,0.6,2.2,11.9,27.4,5,3],None,None,None,None
7,Advertising,marketing,promotions,public relations,and sales managers,11-2000,Summary,6.0,0.1,0.8,7.9,0.1,1.0,1.9,31.1,7,2]
8,Advertising and promotions managers,11-2011,Line Item,0.1,0.0,0.4,0.1,0.0,0.5,0.0,23.0,8,3],None,None,None,None
9,Marketing and sales managers,11-2020,Summary,4.8,0.1,0.7,6.3,0.1,0.9,1.5,31.0,9,3],None,None,None,None


## Creating headers for our DataFrame

#### We're missing headers for our dataframe! How are we even supposed to know what we're looking at?! Luckily, as stated above html has tags to label headers. In our case, it's 'th'. Let's use the same process we did before to get the table cells for getting the table headers

In [8]:
col_headers = text_soup.find_all('th')

all_header = []
col_str = str(col_headers)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)
print(all_header)

['[Occupation Title, Occupation Code, Occupation Type, 2018 Employment, 2018 Percent of Industry, 2018 Percent of Occupation, Projected 2028 Employment, Projected 2028 Percent of Industry, Projected 2028 Percent of Occupation, Employment Change, 2018-2028, Employment Percent Change, 2018-2028, Occupation Sort, Display Level]']


#### Create the dataframe and clean up the output from the for loop

In [9]:
df_headers = pd.DataFrame(all_header)
df_headers.head()

df_headers_clean = df_headers[0].str.split(',', expand=True)
df_headers_clean.head()

df_headers_clean[0] = df_headers_clean[0].str.strip('[')
df_headers_clean.head(10)

df_headers_clean[0] = df_headers_clean[0].str.strip('')
df_headers_clean.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Occupation Title,Occupation Code,Occupation Type,2018 Employment,2018 Percent of Industry,2018 Percent of Occupation,Projected 2028 Employment,Projected 2028 Percent of Industry,Projected 2028 Percent of Occupation,Employment Change,2018-2028,Employment Percent Change,2018-2028,Occupation Sort,Display Level]


### Let's concatenate the two tables we made using concat()

In [10]:
frames = [df_headers_clean, df1]

df4 = pd.concat(frames)
df4.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Occupation Title,Occupation Code,Occupation Type,2018 Employment,2018 Percent of Industry,2018 Percent of Occupation,Projected 2028 Employment,Projected 2028 Percent of Industry,Projected 2028 Percent of Occupation,Employment Change,2018-2028,Employment Percent Change,2018-2028,Occupation Sort,Display Level],NaN,NaN
0,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,\nFilter by Title:\n\n,\nFilter by Code:\n\n,,,,,,,,,,,],None,None,None,None
2,Total,all occupations,00-0000,Summary,7,498.9,100.0,4.7,9,383.1,100.0,5.5,1,884.2,25.1,,0]
3,Management occupations,11-0000,Summary,223.7,3.0,2.2,300.6,3.2,2.8,76.8,34.3,2,1],None,None,None,None
4,Top executives,11-1000,Summary,49.4,0.7,1.8,61.8,0.7,2.2,12.4,25.2,3,2],None,None,None,None
5,Chief executives,11-1011,Line Item,5.8,0.1,2.2,6.3,0.1,2.5,0.5,8.7,4,3],None,None,None,None
6,General and operations managers,11-1021,Line Item,43.5,0.6,1.8,55.5,0.6,2.2,11.9,27.4,5,3],None,None,None,None
7,Advertising,marketing,promotions,public relations,and sales managers,11-2000,Summary,6.0,0.1,0.8,7.9,0.1,1.0,1.9,31.1,7,2]
8,Advertising and promotions managers,11-2011,Line Item,0.1,0.0,0.4,0.1,0.0,0.5,0.0,23.0,8,3],None,None,None,None


#### Based on the dataframe above, we still don't have the appropriate column headers, so let's change that. 

In [11]:
df5 = df4.rename(columns=df4.iloc[0])
df5.head()

,Occupation Title,Occupation Code,Occupation Type,2018 Employment,2018 Percent of Industry,2018 Percent of Occupation,Projected 2028 Employment,Projected 2028 Percent of Industry,Projected 2028 Percent of Occupation,Employment Change,2018-2028,Employment Percent Change,2018-2028,Occupation Sort,Display Level],nan,nan
0,Occupation Title,Occupation Code,Occupation Type,2018 Employment,2018 Percent of Industry,2018 Percent of Occupation,Projected 2028 Employment,Projected 2028 Percent of Industry,Projected 2028 Percent of Occupation,Employment Change,2018-2028,Employment Percent Change,2018-2028,Occupation Sort,Display Level],NaN,NaN
0,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,\nFilter by Title:\n\n,\nFilter by Code:\n\n,,,,,,,,,,,],None,None,None,None
2,Total,all occupations,00-0000,Summary,7,498.9,100.0,4.7,9,383.1,100.0,5.5,1,884.2,25.1,,0]
3,Management occupations,11-0000,Summary,223.7,3.0,2.2,300.6,3.2,2.8,76.8,34.3,2,1],None,None,None,None


#### Next, let's get rid of the first three rows since they do not have anything of value

In [12]:
df6 = df5.drop(df5.index[:3])
df6.head()

,Occupation Title,Occupation Code,Occupation Type,2018 Employment,2018 Percent of Industry,2018 Percent of Occupation,Projected 2028 Employment,Projected 2028 Percent of Industry,Projected 2028 Percent of Occupation,Employment Change,2018-2028,Employment Percent Change,2018-2028,Occupation Sort,Display Level],nan,nan
2,Total,all occupations,00-0000,Summary,7,498.9,100.0,4.7,9,383.1,100.0,5.5,1,884.2,25.1,,0]
3,Management occupations,11-0000,Summary,223.7,3.0,2.2,300.6,3.2,2.8,76.8,34.3,2,1],None,None,None,None
4,Top executives,11-1000,Summary,49.4,0.7,1.8,61.8,0.7,2.2,12.4,25.2,3,2],None,None,None,None
5,Chief executives,11-1011,Line Item,5.8,0.1,2.2,6.3,0.1,2.5,0.5,8.7,4,3],None,None,None,None
6,General and operations managers,11-1021,Line Item,43.5,0.6,1.8,55.5,0.6,2.2,11.9,27.4,5,3],None,None,None,None


#### Finally, let's get rid of the 1 million null columns that we have 

In [13]:
df7 = df6.dropna(axis=1, how='any')

In [14]:
df7

,Occupation Title,Occupation Code,Occupation Type,2018 Employment,2018 Percent of Industry,2018 Percent of Occupation,Projected 2028 Employment,Projected 2028 Percent of Industry,Projected 2028 Percent of Occupation,Employment Change,2018-2028,Employment Percent Change,2018-2028
2,Total,all occupations,00-0000,Summary,7,498.9,100.0,4.7,9,383.1,100.0,5.5,1
3,Management occupations,11-0000,Summary,223.7,3.0,2.2,300.6,3.2,2.8,76.8,34.3,2,1]
4,Top executives,11-1000,Summary,49.4,0.7,1.8,61.8,0.7,2.2,12.4,25.2,3,2]
5,Chief executives,11-1011,Line Item,5.8,0.1,2.2,6.3,0.1,2.5,0.5,8.7,4,3]
6,General and operations managers,11-1021,Line Item,43.5,0.6,1.8,55.5,0.6,2.2,11.9,27.4,5,3]
7,Advertising,marketing,promotions,public relations,and sales managers,11-2000,Summary,6.0,0.1,0.8,7.9,0.1,1.0
8,Advertising and promotions managers,11-2011,Line Item,0.1,0.0,0.4,0.1,0.0,0.5,0.0,23.0,8,3]
9,Marketing and sales managers,11-2020,Summary,4.8,0.1,0.7,6.3,0.1,0.9,1.5,31.0,9,3]
10,Marketing managers,11-2021,Line Item,3.0,0.0,1.1,3.9,0.0,1.4,0.9,30.3,10,4]
11,Sales managers,11-2022,Line Item,1.8,0.0,0.5,2.4,0.0,0.6,0.6,32.1,11,4]


In [35]:
df8 = df7.drop(columns = [' Required', ' Concept',' Attributes',' Limit',' Predicate Type',' Group]'])

In [36]:
df8

,Name,Label
2,AIANHH,American Indian Area/Alaska Native Area/Hawai...
3,AIHHTLI,American Indian Area (Off-Reservation Trust L...
4,AITSCE,American Indian Tribal Subdivision (Census)
5,ANRC,Alaska Native Regional Corporation
6,CBSA,Metropolitan Statistical Area/Micropolitan St...
7,CD113,Congressional District (113th)
8,CNECTA,Combined New England City and Town Area
9,CONCIT,Consolidated City
10,COUNTY,County (FIPS)
11,COUSUB,County Subdivision (FIPS)


In [37]:
list(df8.columns.values)

['Name', ' Label']

In [ ]:
df8.to_csv('CSV_Name_Here')

## Using this script to rename your own columns in a dataframe

There are a couple methods that we can use to 

In [41]:
df8[1250:1300]

,Name,Label
1252,S0501_C03_005E,Foreign born!!Estimate!!5 to 17 years
1253,S0501_C03_006E,Foreign born!!Estimate!!18 to 24 years
1254,S0501_C03_007E,Foreign born!!Estimate!!25 to 44 years
1255,S0501_C03_008E,Foreign born!!Estimate!!45 to 54 years
1256,S0501_C03_009E,Foreign born!!Estimate!!55 to 64 years
1257,S0501_C03_010E,Foreign born!!Estimate!!65 to 74 years
1258,S0501_C03_011E,Foreign born!!Estimate!!75 to 84 years
1259,S0501_C03_012E,Foreign born!!Estimate!!85 years and over
1260,S0501_C03_013E,Foreign born!!Estimate!!Median age (years)
1261,S0501_C03_014E,Foreign born!!Estimate!!RACE AND HISPANIC OR ...
